In [118]:
import pandas as pd
import itertools
import numpy as np

In [119]:
#Read in CSV file with coding
df = pd.read_csv("Final Coding.csv",keep_default_na=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1129 entries, 0 to 1128
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Unnamed: 0                    1129 non-null   int64 
 1   tweet_id                      1129 non-null   int64 
 2   user_username                 1129 non-null   object
 3   text                          1129 non-null   object
 4   created_at                    1129 non-null   object
 5   conversation_id               1129 non-null   int64 
 6   lang                          1129 non-null   object
 7   possibly_sensitive            1129 non-null   bool  
 8   author_id                     1129 non-null   int64 
 9   source                        1129 non-null   object
 10  in_reply_to_user_id           1129 non-null   object
 11  user_verified                 1129 non-null   bool  
 12  user_profile_image_url        1129 non-null   object
 13  user_name         

In [120]:
df = pd.read_csv("Final Coding.csv",keep_default_na=False)
#The codes were in 5 different columns; lets aggregate them into a new column
df["all codes"] = df[["SO code","BC code","BtC code","CCM code","SJEDI code"]].agg(';'.join,axis=1)
#We separated them by ;, so convert to an array 
df["all codes"] = df["all codes"].apply(lambda s: s.split(';'))
#Strip out leading/trailing spaces
df["all codes"] = df["all codes"].apply(lambda s: [t.strip() for t in s if t])
#Sort the arrays, so we can be confident the same things are not listed in two different ways
df["all codes"] = df["all codes"].apply(sorted)
#Convert back to string with ; between the codes
df["all codes"] = df["all codes"].apply(lambda s: ';'.join(s))

df_names = pd.read_csv("Names.csv")
df = df.merge(df_names,left_on='user_username',right_on='User Name',how='left')
df=df.rename(columns={'In JMM 2020 Program?': 'JMM 2020', 'In JMM 2021 Program': 'JMM 2021', 'Too anonymous to determine':'Anonymous'})

print(list(df.columns))
#df[['User Name', 'Display Name', 'JMM 2020', 'JMM 2021', 'Anonymous']]
users = df[['User Name', 'Display Name', 'author_id','JMM 2020','JMM 2021', 'Anonymous', 'Organization']].groupby(by=['User Name']).head(1)
users

#Check merge
print('There were ' + str(len(df['Display Name'].unique())) + ' users in the dataset')
df.to_csv("all-data.csv")

#users

['Unnamed: 0', 'tweet_id', 'user_username', 'text', 'created_at', 'conversation_id', 'lang', 'possibly_sensitive', 'author_id', 'source', 'in_reply_to_user_id', 'user_verified', 'user_profile_image_url', 'user_name', 'user_protected', 'user_created_at', 'user_location', 'user_description', 'user_url', 'user_pinned_tweet_id', 'retweet_count', 'like_count', 'quote_count', 'user_tweet_count', 'user_list_count', 'user_followers_count', 'user_following_count', 'sourcetweet_type', 'sourcetweet_id', 'sourcetweet_text', 'Code', 'sourcetweet_lang', 'sourcetweet_author_id', 'Self-organization', 'SO code', 'Building community', 'BC code', 'Broadening the counterpublic', 'BtC code', 'Creating change in math', 'CCM code', 'SJEDI', 'SJEDI code', 'CODER', 'all codes', 'User Name', 'Display Name', 'JMM 2020', 'JMM 2021', 'Anonymous', 'Organization']
There were 327 users in the dataset


## Export a weighted edge list so Carrie can play around in R with it

## Generate detailed edge list for people network

In [125]:
codes = df['all codes'].apply(lambda s: s.split(';')).explode().unique()
for code in codes:
    df[code] = df['all codes'].apply(lambda s: code in s)

df.head()

users=df[['user_username','author_id']].drop_duplicates()
#users.head()
user_dict = dict(zip(users['author_id'],users['user_username']))

tweet_dict=dict(zip(df['tweet_id'],df['user_username']))


df['Reply']=np.where(df['in_reply_to_user_id']=='NA',False,True)
df['Quote']=np.where(df['sourcetweet_type']=='quoted',True,False)
df['Retweet']=np.where(df['sourcetweet_type']=='retweeted',True,False)

def tweeter_lookup(tweetid):
    print("Looking up tweet "+tweetid)
    if tweetid == "":
        return ""
    elif int(tweetid) in tweet_dict.keys():
        return tweet_dict[int(tweetid)]
    else:
        return ""
    
print(tweet_dict)

#df.loc[df['in_reply_to_user_id']!='NA']
#df.loc[df['Reply']]
df['Receiver']=''
df['Receiver'].mask(df['in_reply_to_user_id']!='NA',df['in_reply_to_user_id'],inplace=True)
#df['Receiver'].mask(df['sourcetweet_type']=='quoted', tweet_dict[df['sourcetweet_id']], inplace=True)
def lookup_username(s):
    if s=='':
        return ''
    elif int(s) not in user_dict.keys():
        return "Error with id "+str(s)
    else:
        return user_dict[int(s)]
df['Receiver'] = df['Receiver'].apply(lookup_username)

df['sourcetweeter']=''
df['sourcetweeter'].mask(df['sourcetweet_type']=='quoted', df['sourcetweet_id'],inplace=True )
df['sourcetweeter'].mask(df['sourcetweet_type']=='retweeted', df['sourcetweet_id'],inplace=True )
df['scourcetweeter']=df['sourcetweeter'].apply(tweeter_lookup)
#df['Sender']=df['sourcetweeter'].apply(lookup_username)
df.head(20)

#users.loc[users['author_id']==31611970]['user_username'].values[0]


{1217856519670288384: 'mathyadriana', 1217856052579971073: 'mathprofcarrie', 1217855474479046656: 'pwr2dppl', 1217854597747875840: 'mathprofcarrie', 1217852288385024000: 'mathprofcarrie', 1217850650438049797: 'xl772', 1217833040866631681: 'WanderingPoint', 1217833038622556161: 'LoboWithACause', 1217812394199531521: 'LoboWithACause', 1217768959518240768: 'mathprofcarrie', 1217757336254799878: 'qsideinstitute', 1217681818268459009: 'frontLahn', 1217674685518733313: 'pwr2dppl', 1217647255571689473: 'RachelleDeCoste', 1217605638756478976: 'mathprofcarrie', 1217600424909238272: 'pwr2dppl', 1217498082822279169: 'pwr2dppl', 1217496966265032705: 'RachelleDeCoste', 1217496159075414017: 'monsoon0', 1217495234164203521: 'elizabethmunch', 1217492198545031169: 'MarissaKawehi', 1217487967738105856: 'pwr2dppl', 1217487540225216514: 'fra_berni', 1217487538660630531: 'HthrLynnJ', 1217485787211665408: 'mathyadriana', 1217485769788612616: 'mathprofcarrie', 1217484892323360774: 'mathprofcarrie', 121748425

,Unnamed: 0,tweet_id,user_username,text,created_at,conversation_id,lang,possibly_sensitive,author_id,source,...,current events,celebration,troll,beyond,Reply,Quote,Retweet,Receiver,sourcetweeter,scourcetweeter
0,1,1217856519670288384,mathyadriana,Anyone feel like looking through the list of i...,2020-01-16T17:09:55.000Z,1217856519670288384,en,False,776450766710136832,Twitter for iPhone,...,False,False,False,False,False,False,False,,,
1,5,1217856052579971073,mathprofcarrie,"""If you think that your right to speech is mor...",2020-01-16T17:08:04.000Z,1217856052579971073,en,False,31611970,Twitter for Android,...,False,False,False,False,False,False,False,,,
2,8,1217855474479046656,pwr2dppl,I ran at elevation to make it to @xl772's #Dis...,2020-01-16T17:05:46.000Z,1217855474479046656,en,False,3308531970,Twitter for Android,...,False,False,False,False,False,False,False,,,
3,9,1217854597747875840,mathprofcarrie,@WanderingPoint @xl772 @pwr2dppl Here!!! #disr...,2020-01-16T17:02:17.000Z,1217850650438049797,en,False,31611970,Twitter for Android,...,False,False,False,False,True,False,False,WanderingPoint,,
4,10,1217852288385024000,mathprofcarrie,@xl772 @pwr2dppl Where and what? - let's get t...,2020-01-16T16:53:06.000Z,1217850650438049797,en,False,31611970,Twitter for Android,...,False,False,False,False,True,False,False,xl772,,
5,12,1217850650438049797,xl772,I’m about to #DisruptJMM in a very small way a...,2020-01-16T16:46:36.000Z,1217850650438049797,en,False,1260729972,Twitter for iPhone,...,False,False,False,False,False,False,False,,,
6,18,1217833040866631681,WanderingPoint,Emerald Stacy starts with a land acknowledgmen...,2020-01-16T15:36:37.000Z,1217833040866631681,en,False,2932698781,Twitter for iPhone,...,False,False,False,False,False,False,False,,,
7,19,1217833038622556161,LoboWithACause,I’m sorry to miss more days of visiting and ev...,2020-01-16T15:36:37.000Z,1217833038622556161,en,False,1969930824,Twitter for iPhone,...,False,False,False,False,False,False,False,,,
8,21,1217812394199531521,LoboWithACause,"Seriosly, taking pictures while accurately and...",2020-01-16T14:14:35.000Z,1217812394199531521,en,False,1969930824,Twitter for iPhone,...,False,False,False,False,False,True,False,,1217492198545031169,MarissaKawehi
9,52,1217768959518240768,mathprofcarrie,My Calling Bull (with R) talk will be TODAY (T...,2020-01-16T11:21:59.000Z,1217768959518240768,en,False,31611970,Twitter Web App,...,False,False,False,False,False,False,False,,,


In [126]:
#f=open("codes edge list.csv","w") 
#Iterate over our code dictionary we created above
#for (codes,weight) in weighted_codes.items():
    #Codes is a string containing the codes separated by ;, so split this into an array of individual codes
#    split_codes = codes.split(";")
#    #We want to turn each hyperedge into an edge for each pair
    #So this iterates over pairs of codes
#    for (c,d) in list(itertools.permutations(split_codes,2)):
        #Put a line in the edgelist for each weight
#        for i in range (0,weight):
#            f.write(c+","+d+"\n")